# **CIS 5450 Final Project - Dog Behavior Analysis**
*Yufa Zhou, Yixiao Ling, Dominik Kau*

#Part 1: Introduction

This project is found in Kaggle competition:\
https://www.kaggle.com/datasets/arashnic/animal-behavior-analysis \
The paper of dataset: \
https://www.sciencedirect.com/science/article/pii/S0168159121001805


In our project, we sought to help dog owners better understand the daily behaviour of their dogs. We used two dataframes. The first dataset 'DogMoveData.csv​' contains the dog movement (accelerometer and gyroscope) data provided by 2 wearable movement sensors (one on the neck and the other one on the back) recorded every 0.01s. The second dataset 'DogInfo.xlsx' contains dogs' biological information, including weight, age months, gender and Neutering Status. The datasets can be accessed via google drive link below:

https://drive.google.com/drive/folders/1-0Y87YFTSeK7agiA_HzI32NqAwzy0_K3?usp=sharing



Aggregating these data, we are able to develop a holistic understanding of both dog movement and dog biological features that affect dog bahavior. Our project problem can be treated as a Time Series Multi-features Classification Problem.

## 1.1 Data Settings

The names of the sensor-data columns are as structured as follows
- A/G: Accelerometer data (acceleration)/Gyroscope data (angular velocity)
- Neck/Back: Sensor mounted on neck/back of dog
- _x/_y/_z: Direction of the quantity

Additionally, the magnitudes of the quanties were calculated whose names follow the same naming convention without the suffix indicating the direction.

DogID: numerical identifier of the dog​

Behavior labels: Dogs are instructed to do several static and dynamic tasks sequentially, alternating between them. Then the actual behavior of the dogs during the assigned tasks were annotated using video recordings. Only segments longer than one second and unambiguous were included in the annotation. ​

Static behaviors: Lying on chest, Sitting, Standing​ \
Dynamic behaviors: Walking, Trotting, Galloping, Sniffing

# Part 2: Data Loading & Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install polars --upgrade

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.5/28.5 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.17.3
    Uninstalling polars-0.17.3:
      Successfully uninstalled polars-0.17.3


In [ ]:
import visualization_functions
from visualization_functions import get_time_plot_frame
from visualization_functions import make_manual_box_plot
import data
import numpy as np
import torch
import sklearn
import polars as pl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time
import sklearn.metrics
from data import TimeSeqDataset
from data import GroupScaler
from data import TimeSeqPCA
from collections import namedtuple
import classification
import plotly.express as px
import plotly

In [ ]:
# get Dataframe
dataset = TimeSeqDataset('./drive/MyDrive/CIS545/545_Final_Project')
df_data = dataset.df_data
behavior_cols = dataset.behavior_cols

# set plotly template and discrete color palette
plotly.io.templates.default = "plotly_white"
color_sequence = px.colors.qualitative.G10

# columns that will be used for group by for anaylsis in scatter plot
group_by_columns = ['Weight', 'Neutered', 'Age months', 'Gender']
# sensor features for which means will be calculated for scatter plots
scatter_sensor_features = ['ABack', 'GBack']

# create exploded Dataframe based on all up to 3 defined behaviors per time step
exploded_df = df_data.explode('Behavior').filter(pl.col('Behavior') != '<undefined>')

# dog behavior that is used for data in scatter plots
scatter_behavior = 'Jumping'
# create pandas DataFrame for scatter plots
df_scatter_plots = exploded_df.filter(pl.col('Behavior')==scatter_behavior).select(
    [*scatter_sensor_features, *group_by_columns]).group_by(group_by_columns).mean().collect().to_pandas()

# include the following features in the analyis of mean
mean_sensor_features = ['ABack', 'GBack', 'ANeck', 'GNeck']
# create pandas DataFrame for plots using mean data
df_mean = exploded_df.select([*mean_sensor_features, 'Behavior']).group_by('Behavior').mean().melt(
    id_vars='Behavior', value_vars=mean_sensor_features).sort('Behavior').collect().to_pandas()

# count the number of occurences of each behavior
df_count = exploded_df.select(['Behavior', 'DogID']).group_by('Behavior').count().sort('Behavior').collect().to_pandas()

# define some axis-labels, that are going to be repeated
mean_labels = {
    'Weight': 'Weight [kg]',
    'ABack': 'Mean acceleration at back [g]',
    'Age months': 'Age [months]',
    'GBack': 'Mean angular velocity at back [°/s]'
}

./drive/MyDrive/CIS545/545_Final_Project/DogInfo.csv


In [ ]:
import pandas as pd

In [ ]:
DogMoveData_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/big data analysis/archive/DogMoveData_csv_format/DogMoveData.csv')
DogMoveData_df.head(10)

,DogID,TestNum,t_sec,ABack_x,ABack_y,ABack_z,ANeck_x,ANeck_y,ANeck_z,GBack_x,GBack_y,GBack_z,GNeck_x,GNeck_y,GNeck_z,Task,Behavior_1,Behavior_2,Behavior_3,PointEvent
0,16,1,0.00,0.041504,0.938965,-0.015137,-0.067871,-0.510254,-0.934570,-17.639161,-22.766115,7.446290,-7.934571,6.347657,13.427735,<undefined>,<undefined>,<undefined>,<undefined>,<undefined>
1,16,1,0.01,0.041992,0.941895,-0.020020,-0.128906,-0.494141,-0.913086,-15.075685,-11.413575,4.821778,-3.906250,4.394532,16.540528,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
2,16,1,0.02,0.040527,0.939453,-0.004395,-0.158691,-0.480469,-0.911133,-12.207032,-0.122070,2.807617,-0.488281,-1.953125,26.794435,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
3,16,1,0.03,0.021484,0.946289,0.007813,-0.122070,-0.486816,-0.880371,-9.460450,7.995606,1.586914,1.159668,-5.676270,38.085940,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
4,16,1,0.04,-0.000977,0.951172,0.033691,-0.053711,-0.500000,-0.807129,-8.361817,14.587403,-1.037598,4.577637,4.089356,41.503909,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
5,16,1,0.05,-0.006348,0.955078,0.038574,0.019043,-0.463867,-0.749023,-9.826661,19.653322,-5.493164,12.939454,20.812990,32.531740,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
6,16,1,0.06,0.001953,0.959473,0.035156,0.045898,-0.376953,-0.781250,-11.657716,15.869142,-7.751465,22.705080,29.968264,14.038087,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
7,16,1,0.07,0.013672,0.958984,0.030273,0.028809,-0.307617,-0.941406,-13.671876,5.859375,-8.361817,23.925783,19.470216,-5.920411,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
8,16,1,0.08,0.028320,0.941895,0.045410,-0.014648,-0.349121,-1.100098,-16.113282,-1.220703,-8.483887,12.084962,-8.300782,-20.202638,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>
9,16,1,0.09,0.028809,0.934082,0.065430,-0.061035,-0.480469,-1.129395,-18.859864,-0.183105,-7.812500,-3.845215,-37.475588,-24.475099,<undefined>,Synchronization,<undefined>,<undefined>,<undefined>


In [ ]:
DogMoveData_df.dtypes

DogID           int64
TestNum         int64
t_sec         float64
ABack_x       float64
ABack_y       float64
ABack_z       float64
ANeck_x       float64
ANeck_y       float64
ANeck_z       float64
GBack_x       float64
GBack_y       float64
GBack_z       float64
GNeck_x       float64
GNeck_y       float64
GNeck_z       float64
Task           object
Behavior_1     object
Behavior_2     object
Behavior_3     object
PointEvent     object
dtype: object

In [ ]:
DogInfo_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/big data analysis/archive/DogMoveData_csv_format/DogInfo.csv')
DogInfo_df.head(10)

,DogID,Breed,Weight,Age months,Gender,NeuteringStatus
0,16,Crossbreed,13,20,1,0
1,18,Belgian Shepherd,29,76,2,0
2,19,German Shepherd,30,57,1,0
3,20,Border Collie,15,47,1,0
4,21,Golden Retriever,39,19,2,0
5,22,Beauceron,35,20,2,0
6,23,Labrador Retriever,23,113,1,1
7,25,Finnish Lapphund,26,88,2,1
8,26,Spanish Water Dog,20,47,2,0
9,27,Mudi,16,99,2,1


In [ ]:
DogInfo_df.dtypes

DogID               int64
Breed              object
Weight              int64
Age months          int64
Gender              int64
NeuteringStatus     int64
dtype: object

#Part 3: Exploratory Data Analysis

After creating summary statistics and understanding the distribution of our data, we decided to further our understanding by evaluating the relationships between meaningful combinations of features. The selection of such combinations was at our discretion, but each combination felt intuitively important -- both from a statistical sense, but also when developing our model.

## 3.1 Dog Behavior class distribution

To begin, we chose to visualize the frequency of each dog behavior class, with each bar stands for the number of times that behavior appears during all time steps of the experiment. This could help us have a general understanding of  the distribution of different behaviors. This also benefits further analysis and modeling, such as combining together rare classes and balancing the class distribution.

In [ ]:
fig = px.bar(
    df_count,
    x='Behavior',
    y='count',
    log_y=False,
    labels={'count':'Frequency'},
    color_discrete_sequence=['green'],
    title='Frequency of the behavior classes'
)

# Update layout for title
fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)

fig.show()

Takeaways:
* 'Lying', 'Pacing', and 'Standing' are the most frequently recorded behaviors, indicating that in the context of the study, these are the most commonly observed states or activities for the dogs.
* The dataset appears to be skewed towards less active behaviors, which could affect any predictive modeling efforts. Notice that we conducted resampling in later modelling section.
* The 'Bowing' behavior is present in only 1540 time steps, compared to over 1.7 million time steps for the 'Panting' behavior.

## 3.2 Distribution of Mean acceleration among all dog behaviors

### 3.2.1 Boxplot of Mean Acceleration

With the above known class distribution of dog behaviors, we decided to figure out the variability and range of acceleration values associated with each behavior. This can show how much movement is typically involved in each behavior and can help in distinguishing between them.

In [ ]:
fig = make_manual_box_plot(
    exploded_df,
    'ABack',
    xlabel='Behavior',
    ylabel='Acceleration at back [g]',
    title="Value distribution of acceleration at back for each behavior"
)

# Update the color of each trace
for i, trace in enumerate(fig.data):
    fig.update_traces(marker=dict(color=color_sequence[i % len(color_sequence)]), selector=i)
# Update layout for title
fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)

fig.show()

Takeaways:
* 'Playing' shows the greatest variation in acceleration values, with a particularly wide interquartile range. This suggests that 'Playing' involves a wide range of movement intensities, which could be vigorous at times.
* 'Sitting' and 'Standing' have relatively tight interquartile ranges, indicating that these behaviors are associated with more consistent and lower levels of acceleration at the back.
* Behaviors such as 'Eating', 'Drinking', 'Sitting', and 'Sniffing' show low median acceleration values and relatively small interquartile ranges. This indicates that these behaviors are associated with low levels of movement at the back, which is consistent with the more stationary nature of these activities. These behaviors are likely characterized by minimal body movement, which could suggest a calmer state compared to those behaviors with higher acceleration values.

### 3.2.2 Histogram of mean acceleration

After the boxplot, we digged deeper into the absolute value of mean acceleration of different dog behaviors. The mean acceleration detected by both sensors (one on the neck and the other one on the back) are both plotted and compared.

In [ ]:
fig = px.bar(
    df_mean[df_mean['variable'].isin(['ABack', 'ANeck'])],
    x='Behavior',
    y='value',
    color='variable',
    labels={**mean_labels, 'value':'Mean acceleration [g]', 'variable': 'Sensor'},
    barmode='group',
    title="Mean acceleration for each behavior",
    color_discrete_sequence=color_sequence
)
# Update layout for title
fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)
fig.show()

Takeaways:
* There is quite simillar (roughly the same) acceleration pattern detected by two sensors.
* The mean acceleration for behaviors like 'Bowing', 'Carrying object', and 'Jumping' is quite consistent between the back and neck sensors . This could imply that these accelerated movements are more uniformly distributed across the dog's body.
* Shaking has notably high mean acceleration values, indicating more vigorous movement.

## 3.3 Distribution of Mean Angular Velocity among all dog behaviors

### 3.3.1 Boxplot of Mean Angular Velocity

We repeated the same visualization procedure for the mean angular velocity.

In [ ]:
fig = make_manual_box_plot(
    exploded_df,
    'GBack',
    xlabel='Behavior',
    ylabel='Angular velocity at back [°/s]',
    title="Value distribution of angular velocity at back for each behavior"
)
fig.update_traces(marker=dict(color=color_sequence[0]))
# Update layout for title
fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)

fig.show()

Takeaways:
* Similarly, the behavior 'Playing' has the highest median angular velocity and also exhibits a broad interquartile range. This suggests that 'Playing' involves rapid changes in direction and possibly vigorous activity that requires substantial body rotation.
* Both 'Walking' and 'Trotting' show moderate median values and interquartile ranges without extreme outliers. This suggests that these behaviors involve consistent and moderate rotational movement, likely due to the rhythmic nature of these gaits.

### 3.3.2 Histogram of mean angular velocity.

Again, we did the same work on the angular velocity.

In [ ]:
fig = px.bar(
    df_mean[df_mean['variable'].isin(['GBack', 'GNeck'])],
    x='Behavior',
    y='value',
    color='variable',
    labels={**mean_labels, 'value':'Mean angular velocity [°/s]', 'variable': 'Sensor'},
    barmode='group',
    title="Mean angular velocity for each behavior",
    color_discrete_sequence=color_sequence,
)
fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)
fig.show()

Takeaways:
* Unlike the acceleration plot, for most behaviors, the mean angular velocity is higher at the back (ABack) than at the neck (ANeck). This could indicate that movements in dogs are more detectable closer to their center of mass.

## 3.4 Feature relationship and correlation

### 3.4.1 Scatter plots of relationship between categorical variables and numerical variables

Then, we investigated the relationship between numerical variables (mean acceleration & mean angular velocity) and categorical variables, Age (x-axis), Weight (size), Gender (fill type) or Neutered status (color). Why we conducted such investigation is that categorical variables might have an influence on the observed behaviors and their associated kinematic patterns. Therefore, by analyzing these relationships, we can identify if certain groups of dogs (like heavier vs. lighter dogs, or males vs. females) exhibit distinct patterns in their movement.

In [ ]:
fig = px.scatter(
    df_scatter_plots,
    x='Age months',
    y='ABack',
    color='Neutered',
    size='Weight',
    symbol='Gender',
    symbol_sequence= ['circle', 'circle-open'],
    title="Mean acceleration on back",
    labels = mean_labels,
    opacity=1,
    color_discrete_sequence=color_sequence
)

fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)

fig.show()

Takeaways:
* We can't see any obvious linear or non-linear relationship between the Age (x-axis), Weight (size), Gender (fill type) or Neutered status (color) for the mean acceleration.

Then, we did the same work on the mean angular velocity and still fail to find any clear linear or non-linear relationship between the dog biological information and mean movement data:

In [ ]:
fig = px.scatter(
    df_scatter_plots,
    x='Age months',
    y='GBack',
    color='Neutered',
    size='Weight',
    symbol='Gender',
    symbol_sequence= ['circle', 'circle-open'],
    title="Mean angular velocity on back",
    labels = mean_labels,
    opacity=1,
    color_discrete_sequence=color_sequence
)
fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)
fig.show()

### 3.4.2 Correlation matrix of numerical variables

We plotted the correlation matrix for all detected sensor data.

In [ ]:
all_sensors = [f'{s}{p}_{d}' for s in ('A', 'G') for p in ('Neck', 'Back') for d in ('x', 'y', 'z')]
correlation_matrix = df_data.select([*all_sensors]).collect().to_pandas().corr()
fig = px.imshow(
    correlation_matrix,
    text_auto='.3f',
    width=800,
    height=800,
    color_continuous_scale='Viridis',
    title='Correlation matrix for sensor data'
)
fig.update_layout(
    title_x=0.3,  # Center title
    title_font=dict(size=25)  # Increase title font size
)
fig.show()

We noticed that there are not any notable linear correlations between the sensor data in x,y,z directions. However, some smaller correlations such as between ABack_y and ANeck_y are visible.

## 3.5 Time series data

### 3.5.1 Time Data for Diffferent Behaviors

Since we have a time-series dataset, we then plotted how acceleration magnitude changes over time for different behaviors. Here, we used an exemplary 2 seconds. By plotting acceleration over time, we can characterize the dynamic nature of each behavior. For example, we can see if a behavior is associated with smooth, consistent movement, or if it is erratic. Also, by understanding what typical acceleration profiles look like for certain behaviors, researchers can check if the sensor data aligns with expected movement patterns and validate sensor data if needed.

In [ ]:
fig = px.line(
    get_time_plot_frame(exploded_df, 2, ['Standing', 'Shaking', 'Jumping'], ['ABack']),
    x = 't_sec',
    y = 'value',
    color = 'variable',
    title = "Acceleration magnitude at back for three behaviors",
    labels={'value':"Acceleration in [g]", 't_sec': "Time [s]", 'variable': 'Behavior'},
    color_discrete_sequence=color_sequence
)
fig.update_layout(
    title_x=0.5,  # Center title
    title_font=dict(size=25)  # Increase title font size
)
fig.show()

Takeaways:
* The 'Shaking' behavior shows very high peaks in acceleration and even gets larger in magnitude over time.
* The 'Jumping' behavior shows a periodic pattern, with regular peaks that likely correspond to the dog's take-off and landing phases.

### 3.5.2 Time Data for Different Sensor Data

Then, we visualized the acceleration in all three axes (x, y, and z) for the 'Shaking' behavior. We plotted the detected acceleration from two sensors (one on the back and the other one on the neck) repectively to see how the movement is distributed across the dog's body. This can inform the best placement of sensors for accurate data collection.

In [ ]:
fig = px.line(
    get_time_plot_frame(exploded_df, 2, ['Shaking'], ['ABack_x', 'ABack_y', 'ABack_z', 'ANeck_x', 'ANeck_y', 'ANeck_z']),
    x = 't_sec',
    y = 'value',
    color = 'variable',
    title = "Acceleration at back in all directions for 'Shaking'",
    labels={'value':"Acceleration in [g]", 't_sec': "Time [s]", 'variable':'Sensor'},
    color_discrete_sequence=color_sequence
)
fig.update_traces(visible='legendonly', selector=dict(name="ANeck_x"))
fig.update_traces(visible='legendonly', selector=dict(name="ANeck_y"))
fig.update_traces(visible='legendonly', selector=dict(name="ANeck_z"))

fig.show()

Takeaways:
* By clicking on different labels on the right of the above diagram, we noticed some lines appearing to mirror or follow a similar pattern. This shows that the sensors have some redundant information. The periodic nature of the movement can be seen for all sensor data, but is most obvious for ABack_x, ANeck_x and ANeck_z.

# Part 4: Modelling

## 4.1 Set (Hyper)Parameters

Here, we define a lot of hyperparameters and settings for the following dataset creation and model training. The most important ones are discussed in the following:
- We found that a time sequence length of 1 s works well. This is a good compromise between having enough information and not having too many features. The time offset of 0.2 s ensures, that we have a large dataset, which is necessary to train neural networks.
- We only used acceleration data which is less noisy than the gyroscope data. Sensor data was included from both the neck and the back and the magnitude as a non-linear feature combination (denoted here without an suffix).
-

In [ ]:
# define some namedtuples for better readability
ModelPerformance = namedtuple('ModelPerformance', "name test_accuracy train_duration")
BatchSizes = namedtuple('BatchSizes', "mlp gru cnn resnet")
EpochNumbers = namedtuple('BatchSizes', "mlp gru cnn resnet")

# path to csv files, NEEDS TO SHOW TO DATA
PATH = './drive/MyDrive/BigDataProject/data'
# length of time-sequence
LEN_SEQ = 100
# offset/overlap between each consecutive time-sequence
SEQ_OFFSET = 0.2
# features used in the time sequence
SENSOR_FEATURES = ['ABack_x', 'ABack_y', 'ABack_z', 'ANeck_x', 'ANeck_y', 'ANeck_z', 'ABack', 'ANeck']
# random seed for functions support it
RANDOM_SEED = 13
# batch sizes for neural network training
BATCH_SIZES = BatchSizes(
    mlp=512,
    gru=512,
    cnn=512,
    resnet=512
)
# number of epochs for neural network training
NUM_EPOCHS = EpochNumbers(
    mlp=50,
    gru=25,
    cnn=50,
    resnet=25
)
# number of epochs for hyperparameter tuning of neural networks
NUM_HYPER_EPOCHS = EpochNumbers(
    mlp=20,
    gru=10,
    cnn=20,
    resnet=10
)
# overwrite 'optimal' device for neural network training
# 'None' if it shouldn't be overwritten
OVERWRITE_DEVICE = None
# columns containing these dog behaviors will be dropped
DROP_MOVEMENT = ('Bowing', 'Panting', 'Carrying object')
# these dog behaviors will be combined
JOIN_MOVEMENTS = (
    ('Drinking', 'Eating'),
    ('Standing', 'Sitting', 'Lying chest'),
    ('Jumping', 'Pacing'),
    ('Playing', 'Tugging')
)
# number of features that will be used after PCA
PCA_NUM_FEATURES = 5
# ratio of the dataset that will be used for testing
TEST_SPLIT = 0.2


model_performances = set()

device = classification.get_device(OVERWRITE_DEVICE)

## 4.2 Create Dataset

In [ ]:
# creating the dataset (this is going to take some time and is CPU intensive)
dataset = TimeSeqDataset(PATH, DROP_MOVEMENT, JOIN_MOVEMENTS)
x, y = dataset.get_numerical_data(SENSOR_FEATURES, LEN_SEQ, SEQ_OFFSET)

# let's plot the number of samples that are present in each class
fig = px.bar(
    x=dataset.class_list,
    y=y.sum(axis=0),
    color_discrete_sequence=color_sequence,
    labels={'x': "Dog movements", 'y':"Frequenccy in dataset"},
    title="Class distribution",
    width=750
)
fig.show()

That doesn't look great. The created dataset has the same issues of class frequency imbalance that was already observed in the EDA phase. We can fix it by oversampling underrepresented classes and undersampling overrepresented classes. However, this should be done after splitting the data into a train and test split, otherwise we duplicate rows could appear in both, the test and train data. That would defeat the purpose of the separation.

We are also scaling the data with a custom class that is adapted from the RobustScaler from sklearn. The RobustScaler minimizes the influence from outliers. The custom adaption ensures, that all acceleration data and all gyroscope data (if any was included) are scaled in the same way respectively. This preserves the ratios of acceleraiton/angular velocity that have a physical meaning.

We also make use of principal component analysis, again with a wrapper around the sklearn class. Here, we want to ensure, that the pca is only applied over the different sensor features, not to each of the 100 time series features. This way we preserve the time sequence information, which is especially important for the CNN and RNN models.

In [ ]:
# split datset into train and test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SPLIT,
                                                    shuffle=True, random_state=RANDOM_SEED)

# scale train and test data, using the RobustScalar on each acceleration
# and gyroscope data seperatly to preserve the ratios between directions
scaler = GroupScaler(SENSOR_FEATURES, LEN_SEQ)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# apply pca to the dataset
pca = TimeSeqPCA(LEN_SEQ)
x_train = pca.fit_transform(x_train, PCA_NUM_FEATURES)
x_test = pca.transform(x_test, PCA_NUM_FEATURES)

# resampling dataset by undersampling classes with more than 25,000 entries
# and oversampling those with less than 25,000 entries
# imbalanced-learn doesn't support multi-label data unfortunately...
# as each sample can have multiple classes, the distribution will not
# necessarily be even after appliying the resample_function once, therefore
# it is executed 5 times here
# including this in our preprocessing increased the accuracy by a lot
for _ in range(5):
    x_train, y_train = classification.resample_data(x_train, y_train, max_count=20_000)
    x_test,  y_test  = classification.resample_data(x_test,  y_test,  max_count=5_000)

# pack training and testing data in tuple for nicer handling
data = (x_train, x_test, y_train, y_test)

In this plot we see the explained variance of the PCA. For five sensor features, we reach a value of 0.8 which is why we chose in the above section.

In [ ]:
fig = px.line(
    x=np.arange(1, len(SENSOR_FEATURES)+1),
    y=np.cumsum(pca.pca.explained_variance_ratio_),
    title="Explained Variance of PCA components",
    labels={'x': "Number of Features", 'y':"Explained Variance"},
    color_discrete_sequence=color_sequence,
    width=750
)
fig.add_hline(y=0.8, line_color='black')
fig.show()

Let's see if the class distribution looks better after the resampling procedure.

In [ ]:
# plot class distribution for training data
fig = px.bar(
    x=dataset.class_list,
    y=y_train.sum(axis=0),
    color_discrete_sequence=color_sequence,
    labels={'x': "Dog movements", 'y':"Frequency in dataset"},
    title="Class distribution of Training Dataset After Resampling",
    width=750
)
fig.show()

# plot class distribution for testing data
fig = px.bar(
    x=dataset.class_list,
    y=y_test.sum(axis=0),
    color_discrete_sequence=color_sequence,
    labels={'x': "Dog movements", 'y':"Frequency in dataset"},
    title="Class distribution of Testing Dataset After Resampling",
    width=750
)
fig.show()

# print out number of samples in training dataset
print()
print(f"Total number of samples in training dataset: {y_train.shape[0]}")

Total number of samples in training dataset: 150372


Yes! That is much nicer than before and in total we have a lot of rows in our dataset.

## 4.3 Baseline Model - Deciscion Tree Classifier

We chose the decision tree classifier as our baseline model because it's inherently categorical and a very simple and easy-to-use model.

### 4.3.1 Hyperparmeter tuning

Here, we optimize the minimum samples per leaf in the final decision tree and the minimum number of samples required for a split.

In [ ]:
# initialize DecisionTree model
model = DecisionTreeClassifier()

# define parameters for grid search
param_grid = {'min_samples_leaf': [8, 16] ,'min_samples_split': [32, 64]}
# use grid search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=4)
grid_search.fit(x_train, y_train)
print(f"Best parameters: {grid_search.best_params_}")

Best parameters: {'min_samples_leaf': 8, 'min_samples_split': 32}


### 4.3.2 Training with Tuned Hyperparameters

In [ ]:
# train the model again on the best hyperparameters
model.set_params(**grid_search.best_params_)
print(f"Starting training Decision Tree")
t0 = time.time()
model.fit(x_train, y_train)
t1 = time.time()
duration = t1-t0
print(f"Training took {duration:.1f} s")

Starting training Decision Tree
Training took 298.0 s


### 4.3.3 Model Evaluation

For multi-label problems we cannot determine a single confusion matrix. Instead, we calculate one confusion matrix for each class.

In [ ]:
# model evaluation on training data
train_accuracy = model.score(x_train, y_train)
print(f"Training accuracy: {train_accuracy*100:.2f} %")
# model evaluation on testing data
test_accuracy = model.score(x_test, y_test)
print(f"Testing accuracy: {test_accuracy*100:.2f} %")

# compute confusion matrix on test data
y_pred = model.predict(x_test)
confusion_matrices = sklearn.metrics.multilabel_confusion_matrix(y_test, y_pred)
# plot the multil-label confusion matrix
classification.plot_confusion_matrices(confusion_matrices, dataset.class_list, title="Confusion matrices for Decision Tree")

# save model performance
model_performances.add(
    ModelPerformance(name='Decision Tree', test_accuracy=test_accuracy, train_duration=duration)
)

Training accuracy: 92.41 %
Testing accuracy: 67.96 %


There's a lot of overfitting with this model. One obvious downside to the decision tree is that the construct of time is not represented in the model.

## 4.5 A Small Upgrade - Random Forest Classifier

Let's try again with one of the most popular categorical predictors.

### 4.5.1 Hyperparmeter tuning

In [ ]:
# initialize DecisionTree model
model = RandomForestClassifier(n_jobs=-1)

# define parameters for grid search
param_grid = {'n_estimators': [50, 100], 'min_samples_split': [32, 64]}
# use grid search to find the best hyper-parameters
grid_search = GridSearchCV(model, param_grid, cv=4)
grid_search.fit(x_train, y_train)
print(f"Best parameters: {grid_search.best_params_}")

Best parameters: {'min_samples_split': 32, 'n_estimators': 50}


### 4.5.2 Training with Tuned Hyperparameters

In [ ]:
# train the model again on the best hyperparameters
model.set_params(**grid_search.best_params_, n_jobs=-1)
print(f"Starting training Random Forest")
t0 = time.time()
model.fit(x_train, y_train)
t1 = time.time()
duration = t1-t0
print(f"Training took {duration:.1f} s")

Starting training Random Forest
Training took 457.5 s


### 4.5.3 Model Evaluation

In [ ]:
# model evaluation on training data
train_accuracy = model.score(x_train, y_train)
print(f"Training accuracy: {train_accuracy*100:.2f} %")
# model evaluation on testing data
test_accuracy = model.score(x_test, y_test)
print(f"Testing accuracy: {test_accuracy*100:.2f} %")

# compute confusion matrix on test data
y_pred = model.predict(x_test)
confusion_matrices = sklearn.metrics.multilabel_confusion_matrix(y_test, y_pred)
# plot the multil-label confusion matrix
classification.plot_confusion_matrices(confusion_matrices, dataset.class_list, title="Confusion matrices for Random Forest")

# save model performance
model_performances.add(
    ModelPerformance(name='Random Forest', test_accuracy=test_accuracy, train_duration=duration)
)

Training accuracy: 97.81 %
Testing accuracy: 73.45 %


## 4.6 A Simple Neural Network - MLP

Now we're moving to the more advanced neural networks. The first one is still relatively simple: a MLP consisting of three fully connected layers.

### 4.6.1 Hyperparameter Tuning

This model tends to overfit, so we're going to tune the weight decay, which will counteract the overfitting tendencies.

In [ ]:
# get training and testing DataLoaders
train_loader, test_loader = classification.get_dataloaders(
    data, 'MLP', LEN_SEQ, BATCH_SIZES.mlp
)
# initialize loss function
# the BCEWithLogitsLoss function includes a sigmoid function which therefore
# doesn't have to be included in the actual model (this is numerially more stable)
loss_fun = torch.nn.BCEWithLogitsLoss()

# define list of variable hyperparameter
weight_decay_list = np.logspace(-4, -2, 4)
# initialize list of test errors
tuning_errors = []

print(f"Starting hyperparameter tuning of MLP model")
for weight_decay in weight_decay_list:
    # initialize MLP model
    model = classification.TorchMLP(
        input_size=x_train.shape[1],
        output_size=y_train.shape[1],
        hidden_layers=(64, 64, 64)
    )
    # initialize optimizer with model parameters
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=weight_decay)
    # train the model
    training_history = classification.train_loop(
        model,
        train_loader,
        test_loader,
        loss_fun,
        optimizer,
        NUM_HYPER_EPOCHS.mlp,
        device
    )
    # save model performance
    tuning_errors.append(training_history.test.error[-1])

# save and print best weight decay
best_weight_decay = weight_decay_list[np.argmin(tuning_errors)]
print(f"Best testing accuracy was achieved with weight decay of {best_weight_decay:.3e}")

Starting hyperparameter tuning of MLP model
Training on CUDA.
Epoch 1/20
  training loss: 0.741894  [  512/150372]
  training loss: 0.404099  [50688/150372]
  training loss: 0.310903  [100864/150372]
 validation error 36.29 %
Epoch 2/20
  training loss: 0.264025  [  512/150372]
  training loss: 0.227708  [50688/150372]
  training loss: 0.190953  [100864/150372]
 validation error 24.86 %
Epoch 3/20
  training loss: 0.159443  [  512/150372]
  training loss: 0.145464  [50688/150372]
  training loss: 0.134845  [100864/150372]
 validation error 21.17 %
Epoch 4/20
  training loss: 0.114644  [  512/150372]
  training loss: 0.099511  [50688/150372]
  training loss: 0.114272  [100864/150372]
 validation error 21.28 %
Epoch 5/20
  training loss: 0.091424  [  512/150372]
  training loss: 0.096620  [50688/150372]
  training loss: 0.080511  [100864/150372]
 validation error 18.72 %
Epoch 6/20
  training loss: 0.084483  [  512/150372]
  training loss: 0.085491  [50688/150372]
  training loss: 0.0772

### 4.6.2 Training with Tuned Hyperparameters

In [ ]:
# initialize MLP model
model = classification.TorchMLP(
    input_size=x_train.shape[1],
    output_size=y_train.shape[1],
    hidden_layers=(64, 64, 64)
)
# initialize optimizer with model parameters and optimized weight decay
optimizer = torch.optim.Adam(model.parameters(), weight_decay=best_weight_decay)

print(f"Starting training MLP model")
t0 = time.time()
training_history = classification.train_loop(
    model,
    train_loader,
    test_loader,
    loss_fun,
    optimizer,
    NUM_EPOCHS.mlp,
    device
)
t1 = time.time()
duration = t1-t0
print(f"Training took {duration:.1f} s")

Starting training MLP model
Training on CUDA.
Epoch 1/50
  training loss: 0.735288  [  512/150372]
  training loss: 0.406926  [50688/150372]
  training loss: 0.315652  [100864/150372]
 validation error 38.34 %
Epoch 2/50
  training loss: 0.268941  [  512/150372]
  training loss: 0.222422  [50688/150372]
  training loss: 0.199045  [100864/150372]
 validation error 26.09 %
Epoch 3/50
  training loss: 0.180719  [  512/150372]
  training loss: 0.150362  [50688/150372]
  training loss: 0.124759  [100864/150372]
 validation error 21.12 %
Epoch 4/50
  training loss: 0.114557  [  512/150372]
  training loss: 0.106864  [50688/150372]
  training loss: 0.092587  [100864/150372]
 validation error 21.25 %
Epoch 5/50
  training loss: 0.134193  [  512/150372]
  training loss: 0.091320  [50688/150372]
  training loss: 0.082713  [100864/150372]
 validation error 18.85 %
Epoch 6/50
  training loss: 0.078657  [  512/150372]
  training loss: 0.060899  [50688/150372]
  training loss: 0.082360  [100864/1503

### 4.6.3 Model Evaluation

In [ ]:
# plot the training and testing accuracies/losses during training
classification.plot_training_run(training_history, title="Training run with MLP model")
# compute confusion matrix on test data
confusion_matrices = classification.calculate_confusion_matrices_torch(model, test_loader, device)
# plot the multil-label confusion matrix
classification.plot_confusion_matrices(confusion_matrices, dataset.class_list, title="Confusion matrices for MLP model")

# save model performance
model_performances.add(
    ModelPerformance(name='MLP', test_accuracy=1-training_history.test.error[-1], train_duration=duration)
)

## 4.7 Sequence Based Neural Network - GRU

This is the first model, that "understands" the time series nature of our data. We're using a GRU instead of a standard recurrent neural network because the sequence length of 100 could cause vanishing or exploding gradients in the latter case.

### 4.7.1 Hyperparameter Tuning

The model has three recurrent layers. In this section the mdoel will be tuned to the ideal number of nodes.

In [ ]:
# get training and testing DataLoaders
train_loader, test_loader = classification.get_dataloaders(
    data, 'GRU', LEN_SEQ, BATCH_SIZES.gru
)
# initialize loss function
loss_fun = torch.nn.BCEWithLogitsLoss()

# define list of variable hyperparameter
hidden_size_list = [48, 64, 96]
# initialize list of test errors
tuning_errors = []

print(f"Starting hyperparameter tuning of GRU model")
for hidden_size in hidden_size_list:
    # initialize GRU model
    model = classification.TorchGRU(
        input_size=PCA_NUM_FEATURES,
        output_size=y.shape[1],
        hidden_size=hidden_size,
        rnn_layers=3
    )
    # initialize optimizer with model parameters
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-4)
    # train the model
    training_history = classification.train_loop(
        model,
        train_loader,
        test_loader,
        loss_fun,
        optimizer,
        NUM_HYPER_EPOCHS.gru,
        device
    )
    # save model performance
    tuning_errors.append(training_history.test.error[-1])

# save and print best hidden size
best_hidden_size = hidden_size_list[np.argmin(tuning_errors)]
print(f"Best testing accuracy was achieved with a hidden size of {best_hidden_size}")

Starting hyperparameter tuning of GRU model
Training on CUDA.
Epoch 1/10
  training loss: 0.687646  [  512/150372]
  training loss: 0.332900  [50688/150372]
  training loss: 0.231718  [100864/150372]
 validation error 58.21 %
Epoch 2/10
  training loss: 0.193023  [  512/150372]
  training loss: 0.163764  [50688/150372]
  training loss: 0.134056  [100864/150372]
 validation error 41.64 %
Epoch 3/10
  training loss: 0.127818  [  512/150372]
  training loss: 0.121795  [50688/150372]
  training loss: 0.129758  [100864/150372]
 validation error 28.68 %
Epoch 4/10
  training loss: 0.108184  [  512/150372]
  training loss: 0.102800  [50688/150372]
  training loss: 0.112982  [100864/150372]
 validation error 25.17 %
Epoch 5/10
  training loss: 0.105656  [  512/150372]
  training loss: 0.100979  [50688/150372]
  training loss: 0.102660  [100864/150372]
 validation error 22.33 %
Epoch 6/10
  training loss: 0.094219  [  512/150372]
  training loss: 0.095909  [50688/150372]
  training loss: 0.0826

### 4.7.2 Training with Tuned Hyperparameters

In [ ]:
# initialize GRU model
model = classification.TorchGRU(
    input_size=PCA_NUM_FEATURES,
    output_size=y.shape[1],
    hidden_size=best_hidden_size,
    rnn_layers=3
)
optimizer = torch.optim.Adam(model.parameters())

print(f"Starting training GRU model")
t0 = time.time()
training_history = classification.train_loop(
    model,
    train_loader,
    test_loader,
    loss_fun,
    optimizer,
    NUM_EPOCHS.gru,
    device
)
t1 = time.time()
duration = t1-t0
print(f"Training took {duration:.1f} s")

Starting training GRU model
Training on CUDA.
Epoch 1/25
  training loss: 0.695695  [  512/150372]
  training loss: 0.230515  [50688/150372]
  training loss: 0.173053  [100864/150372]
 validation error 32.48 %
Epoch 2/25
  training loss: 0.131668  [  512/150372]
  training loss: 0.117996  [50688/150372]
  training loss: 0.097078  [100864/150372]
 validation error 23.64 %
Epoch 3/25
  training loss: 0.090202  [  512/150372]
  training loss: 0.085184  [50688/150372]
  training loss: 0.089054  [100864/150372]
 validation error 16.90 %
Epoch 4/25
  training loss: 0.071671  [  512/150372]
  training loss: 0.050906  [50688/150372]
  training loss: 0.066371  [100864/150372]
 validation error 14.40 %
Epoch 5/25
  training loss: 0.054539  [  512/150372]
  training loss: 0.046220  [50688/150372]
  training loss: 0.046028  [100864/150372]
 validation error 12.04 %
Epoch 6/25
  training loss: 0.050245  [  512/150372]
  training loss: 0.051994  [50688/150372]
  training loss: 0.052398  [100864/1503

### 4.7.3 Model Evaluation

In [ ]:
# plot the training and testing accuracies/losses during training
classification.plot_training_run(training_history, title="Training run with GRU model")
# compute confusion matrix on test data
confusion_matrices = classification.calculate_confusion_matrices_torch(model, test_loader, device)
# plot the multil-label confusion matrix
classification.plot_confusion_matrices(confusion_matrices, dataset.class_list, title="Confusion matrices for GRU model")

# save model performance
model_performances.add(
    ModelPerformance(name='GRU', test_accuracy=1-training_history.test.error[-1], train_duration=duration)
)

## 4.8 Different Kind of Sequence Analysis - Simple CNN

1D-convolutional neural networks can also be used to analyze time series data. This CNN has a relatively simple structure.

### 4.8.1 Hyperparameter Tuning

In this case we want to tune the number of elements in the fully connected layer.

In [ ]:
# get training and testing DataLoaders
train_loader, test_loader = classification.get_dataloaders(
    data, 'CNN', LEN_SEQ, BATCH_SIZES.cnn
)
# initialize loss function
loss_fun = torch.nn.BCEWithLogitsLoss()

# define list of variable hyperparameter
hidden_size_list = [64, 96, 128]
# initialize list of test errors
tuning_errors = []

print(f"Starting hyperparameter tuning of CNN model")
for hidden_size in hidden_size_list:
    # initialize the CNN1D model
    model = classification.CNN1D(
        input_size=PCA_NUM_FEATURES,
        output_size=y_train.shape[1],
        hidden_size=hidden_size
    )
    # initialize optimizer with model parameters
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-5)
    # train the model
    training_history = classification.train_loop(
        model,
        train_loader,
        test_loader,
        loss_fun,
        optimizer,
        NUM_HYPER_EPOCHS.cnn,
        device
    )
    # save model performance
    tuning_errors.append(training_history.test.error[-1])

# save and print best hidden size
best_hidden_size = hidden_size_list[np.argmin(tuning_errors)]
print(f"Best testing accuracy was achieved with a hidden size of {best_hidden_size}")

Starting hyperparameter tuning of CNN model
Training on CUDA.
Epoch 1/20
  training loss: 0.744133  [  512/150372]
  training loss: 0.363131  [50688/150372]
  training loss: 0.186028  [100864/150372]
 validation error 34.21 %
Epoch 2/20
  training loss: 0.141535  [  512/150372]
  training loss: 0.124626  [50688/150372]
  training loss: 0.117054  [100864/150372]
 validation error 27.85 %
Epoch 3/20
  training loss: 0.101197  [  512/150372]
  training loss: 0.106266  [50688/150372]
  training loss: 0.099836  [100864/150372]
 validation error 25.21 %
Epoch 4/20
  training loss: 0.092373  [  512/150372]
  training loss: 0.093442  [50688/150372]
  training loss: 0.085921  [100864/150372]
 validation error 22.06 %
Epoch 5/20
  training loss: 0.089710  [  512/150372]
  training loss: 0.085132  [50688/150372]
  training loss: 0.081700  [100864/150372]
 validation error 20.77 %
Epoch 6/20
  training loss: 0.090460  [  512/150372]
  training loss: 0.073950  [50688/150372]
  training loss: 0.0719

### 4.8.2 Training with Tuned Hyperparameters

In [ ]:
# initialize CNN1D model
model = classification.CNN1D(
    input_size=PCA_NUM_FEATURES,
    output_size=y_train.shape[1],
    hidden_size=best_hidden_size
)
loss_fun = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

print(f"Starting training CNN model")
t0 = time.time()
training_history = classification.train_loop(
    model,
    train_loader,
    test_loader,
    loss_fun,
    optimizer,
    NUM_EPOCHS.cnn,
    device
)
t1 = time.time()
duration = t1-t0
print(f"Training took {duration:.1f} s")

Starting training CNN model
Training on CUDA.
Epoch 1/50
  training loss: 0.751556  [  512/150372]
  training loss: 0.309279  [50688/150372]
  training loss: 0.151352  [100864/150372]
 validation error 29.48 %
Epoch 2/50
  training loss: 0.131266  [  512/150372]
  training loss: 0.103036  [50688/150372]
  training loss: 0.082245  [100864/150372]
 validation error 24.59 %
Epoch 3/50
  training loss: 0.090126  [  512/150372]
  training loss: 0.096793  [50688/150372]
  training loss: 0.094297  [100864/150372]
 validation error 21.92 %
Epoch 4/50
  training loss: 0.079748  [  512/150372]
  training loss: 0.096808  [50688/150372]
  training loss: 0.078807  [100864/150372]
 validation error 21.44 %
Epoch 5/50
  training loss: 0.088303  [  512/150372]
  training loss: 0.073167  [50688/150372]
  training loss: 0.074289  [100864/150372]
 validation error 19.63 %
Epoch 6/50
  training loss: 0.074487  [  512/150372]
  training loss: 0.081928  [50688/150372]
  training loss: 0.070138  [100864/1503

### 4.8.3 Model Evaluation

In [ ]:
# plot the training and testing accuracies/losses during training
classification.plot_training_run(training_history, title="Training run with CNN model")
# compute confusion matrix on test data
confusion_matrices = classification.calculate_confusion_matrices_torch(model, test_loader, device)
# plot the multil-label confusion matrix
classification.plot_confusion_matrices(confusion_matrices, dataset.class_list, title="Confusion matrices for CNN model")

# save model performance
model_performances.add(
    ModelPerformance(name='CNN', test_accuracy=1-training_history.test.error[-1], train_duration=duration)
)

## 4.9 A Fancy CNN - ResNet

Here, we build build a more complex convolutional network. The model is based on the ResNet architecture (see: https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py).

### 4.9.1 Hyperparameter Tuning

In [ ]:
# get training and testing DataLoaders
train_loader, test_loader = classification.get_dataloaders(
    data, 'CNN', LEN_SEQ, BATCH_SIZES.resnet
)
# initialize loss function
loss_fun = torch.nn.BCEWithLogitsLoss()

# define list of variable hyperparameter
layer_sizes_list = [[2, 2, 2, 2], [2, 3, 3, 2], [2, 3, 4, 2]]
# initialize list of test errors
tuning_errors = []

print(f"Starting hyperparameter tuning of CNN Resnet model")
for layer_sizes in layer_sizes_list:
    # initialize the ResNet1D model
    model = classification.ResNet1D(
        input_size=PCA_NUM_FEATURES,
        output_size=y.shape[1],
        block=classification.ResidualBlock,
        layers=layer_sizes
    )
    # initialize optimizer with model parameters
    optimizer = torch.optim.AdamW(model.parameters())
    # train the model
    training_history = classification.train_loop(
        model,
        train_loader,
        test_loader,
        loss_fun,
        optimizer,
        NUM_HYPER_EPOCHS.resnet,
        device
    )
    # save model performance
    tuning_errors.append(training_history.test.error[-1])

# save and print best layer sizes
best_layer_sizes = layer_sizes_list[np.argmin(tuning_errors)]
print(f"Best testing accuracy was achieved with layer sizes of {best_layer_sizes}")

Starting hyperparameter tuning of CNN Resnet model
Training on CUDA.
Epoch 1/10
  training loss: 0.670064  [  512/150372]
  training loss: 0.073881  [50688/150372]
  training loss: 0.036186  [100864/150372]
 validation error 13.68 %
Epoch 2/10
  training loss: 0.043360  [  512/150372]
  training loss: 0.024886  [50688/150372]
  training loss: 0.028340  [100864/150372]
 validation error 9.75 %
Epoch 3/10
  training loss: 0.025735  [  512/150372]
  training loss: 0.019905  [50688/150372]
  training loss: 0.024204  [100864/150372]
 validation error 9.39 %
Epoch 4/10
  training loss: 0.016568  [  512/150372]
  training loss: 0.015565  [50688/150372]
  training loss: 0.013219  [100864/150372]
 validation error 8.75 %
Epoch 5/10
  training loss: 0.018639  [  512/150372]
  training loss: 0.016853  [50688/150372]
  training loss: 0.013634  [100864/150372]
 validation error 9.51 %
Epoch 6/10
  training loss: 0.008674  [  512/150372]
  training loss: 0.013209  [50688/150372]
  training loss: 0.0

### 4.9.2 Training with Tuned Hyperparameters

In [ ]:
# initialize ResNet1D model
model = classification.ResNet1D(
    input_size=PCA_NUM_FEATURES,
    output_size=y.shape[1],
    block=classification.ResidualBlock,
    layers=best_layer_sizes
)
loss_fun = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters())

print(f"Starting training CNN Resnet model")
t0 = time.time()
training_history = classification.train_loop(
    model,
    train_loader,
    test_loader,
    loss_fun,
    optimizer,
    NUM_EPOCHS.resnet,
    device
)
t1 = time.time()
duration = t1-t0
print(f"Training took {duration:.1f} s")

Starting training CNN Resnet model
Training on CUDA.
Epoch 1/25
  training loss: 0.718627  [  512/149100]
  training loss: 0.062077  [50176/149100]
  training loss: 0.056955  [99840/149100]
  training loss: 0.033549  [149100/149100]
 validation error 11.94 %
Epoch 2/25
  training loss: 0.044491  [  512/149100]
  training loss: 0.030470  [50176/149100]
  training loss: 0.024754  [99840/149100]
  training loss: 0.035084  [149100/149100]
 validation error 11.36 %
Epoch 3/25
  training loss: 0.030591  [  512/149100]
  training loss: 0.023465  [50176/149100]
  training loss: 0.024262  [99840/149100]
  training loss: 0.040865  [149100/149100]
 validation error 15.49 %
Epoch 4/25
  training loss: 0.021651  [  512/149100]
  training loss: 0.016306  [50176/149100]
  training loss: 0.012430  [99840/149100]
  training loss: 0.024835  [149100/149100]
 validation error 10.42 %
Epoch 5/25
  training loss: 0.015147  [  512/149100]
  training loss: 0.021726  [50176/149100]
  training loss: 0.014361  [

### 4.9.3 Model Evaluation

In [ ]:
# plot the training and testing accuracies/losses during training
classification.plot_training_run(training_history, title="Training run with CNN Resnet model")
# compute confusion matrix on test data
confusion_matrices = classification.calculate_confusion_matrices_torch(model, test_loader, device)
# plot the multil-label confusion matrix
classification.plot_confusion_matrices(confusion_matrices, dataset.class_list, title="Confusion matrices for CNN Resnet model")

# save model performance
model_performances.add(
    ModelPerformance(name='CNN Resnet', test_accuracy=1-training_history.test.error[-1], train_duration=duration)
)

## 4.10 Model Conclusion

First, let's print the accuracies of all the models in one place.

In [ ]:
for performance in model_performances:
    print(f"====== {performance.name} ======")
    print(f"  Test accuracy: {performance.test_accuracy*100:.2f} %")
    print(f"  Training time: {performance.train_duration:.1f} s")

====== CNN Resnet ======
  Test accuracy: 92.40 %
  Training time: 663.4 s
====== CNN ======
  Test accuracy: 85.80 %
  Training time: 183.7 s
====== Decision Tree ======
  Test accuracy: 67.96 %
  Training time: 298.0 s
====== Random Forest ======
  Test accuracy: 73.45 %
  Training time: 457.5 s
====== MLP ======
  Test accuracy: 83.16 %
  Training time: 162.6 s
====== GRU ======
  Test accuracy: 92.51 %
  Training time: 307.9 s


We can clearly see, that the two best models are the CNN Resnet and the GRU. Both are architectures, that inherently model sequences. Our maximum accuracy is higher than the accuracy achieved by the paper in which the data was orginially published.

# Part 5: Conclusion

**Our project can benefit dog owners:**

* Health Monitoring: The correlation between weight and acceleration magnitude could be used to monitor a dog's health and fitness. A sudden change in this relationship might indicate health issues.​

* Tailored Exercise: Owners can tailor exercise to suit their dog's weight and age to ensure their pet is getting appropriate levels of activity.

**Overall Experience + Team Reflection:**

* It is critical to resample dataset to get a uniform data distribution. If we just use the original dataset, the accuracy will be stuck around 80%.​

* Hands-on data analysis experience help a lot, e.g. BatchNorm is very important; otherwise the model will always have very low accuracy.